In [1]:
# NLP 기반 데이터 분석 및 머신러닝/딥러닝 모델
# 1. AI 허브, 공공정보데이터 => 데이터 전처리(라벨링)
# 2. 한국어 처리를 위한 준비
#     A. 불용어 처리 (용어 리스트) / 단어뭉치 (세종 코퍼스)
#     B. 토큰분리기 (Mecab(리눅스에서 돌아감)or KoNLP 써야 함)

NLP 기반 데이터 분석 및 머신러닝/딥러닝 모델
1. AI 허브, 공공정보데이터 => 데이터 전처리(라벨링)
2. 한국어 처리를 위한 준비
     - A. 불용어 처리 (용어 리스트) / 단어뭉치 (세종 코퍼스)
     - B. 토큰분리기 (Mecab(리눅스에서 돌아감)or KoNLP 써야 함)

# 네이버 영화 댓글을 활용한 감정분석
1. 데이터를 기반으로 지도 알고리즘을 선별
2. NLP의 특성상 토큰 분리에 대한 기준을 설정
3. 모델 #1 학습 후 추출(dump)
## 모델 로드 후 테스트
1. "pkl" 파일을 불러오기
2. "전처리 과정을 그대로 복제"
3. 이후 추론

In [2]:
import numpy as np
import pandas as pd

In [3]:
train_df = pd.read_table("data/ratings_train.txt")
test_df = pd.read_table("data/ratings_test.txt")

In [4]:
train_df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [5]:
# Null 및 숫자를 공백 문자로 변경
# train_df["document"]
# 가능하면 축약 리스트 활용
# 가능하면 for를 쓰지 않는 방향으로 진행
import re
train_df = train_df.fillna(" ")
train_df["document"] = train_df["document"].apply(lambda x: re.sub(r"\d+", " ", x))
test_df =test_df.fillna(" ")
test_df["document"] = test_df["document"].apply(lambda x : re.sub(r"\d+", " ", x))


In [6]:
from konlpy.tag import Okt
okt = Okt()

In [11]:
def tw_tokenizer(text):
    tokenizer_ko = okt.morphs(text)
    return tokenizer_ko

In [12]:
tw_tokenizer("가난한 내가 나타샤를 사랑해서")

['가난한', '내', '가', '나타', '샤를', '사랑', '해서']

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(tokenizer=tw_tokenizer,
                        ngram_range=(1, 2),
                        min_df=3,
                        max_df=0.9)

In [14]:
tfidf.fit(train_df["document"])

c:\Users\uear\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.9, min_df=3, ngram_range=(1, 2),
                tokenizer=<function tw_tokenizer at 0x000002487C1941F0>)

In [15]:
tfidf_matrix_train = tfidf.transform(train_df["document"])
tfidf_matrix_train

<150000x129276 sparse matrix of type '<class 'numpy.float64'>'
	with 3069141 stored elements in Compressed Sparse Row format>

In [16]:
# 텍스트->희소행렬로 할 때 tfidf 인 이유 : 영화에 한정된 데이터이기 때문에
# 만약 kakao톡같은 경우에는 vetorizer 쓰는 게 훨 낫다고 함
from sklearn.linear_model import LogisticRegression
lg_clf = LogisticRegression(C=3.5, random_state=42)
lg_clf.fit(tfidf_matrix_train, train_df["label"])

LogisticRegression(C=3.5, random_state=42)

In [18]:
import joblib

In [19]:
joblib.dump(lg_clf, "movie_lr.pkl")

['movie_lr.pkl']

In [20]:
joblib.dump(tfidf, "movie_lr_tfidf.pkl")

['movie_lr_tfidf.pkl']

## 역직렬화

In [21]:
movie_lr = joblib.load("movie_lr.pkl")
movie_lr_tfidf = joblib.load("movie_lr_tfidf.pkl")

In [39]:
q = "재밌어요"
q = re.sub(r"d\+", " ", q)
q_matrix = movie_lr_tfidf.transform([q])

In [40]:
movie_lr.predict(q_matrix)

array([1], dtype=int64)